# Create a family knowledge base of the primitive relations: Parent(Parent, Child), Male(Person), Female(Person), and Married(Person, Person). Define a set of 10 relations (for example: Brother, Sister, Chacha, Chachi, Cousin...). Write a program to interactively answer the following kind of queries –“How is X related to Y”. How will you generate the shortest possible chain of relations? For example “Ramesh is Sunita’s maami’s cousin”.


In [35]:
primitive_relations =['Male','Female','Married','Parent']
relation = ['Sibling','Dad','Mom','Cousin','Spouse','Uncle','Aunt','Grandparent','Grandchild']
print(len(relation))

9


In [3]:
def parse_family_relations_v3(file_path):
    relation_based_dict = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Ensure the line is not empty
                # Extracting the relation type and the individuals involved
                relation, people = line.strip().split('(')
                person1, person2 = people.strip(')').split(',')
                
                # Update the dictionary for the relation
                if relation not in relation_based_dict:
                    relation_based_dict[relation] = []
                relation_based_dict[relation].append([person1, person2])
                
                # Handle bidirectional relations such as 'Married'
                if relation == 'Married':
                    # For 'Married', we've already added both persons in the right order
                    # If other bidirectional relations need to be handled, similar logic could be applied here
                    pass
    
    return relation_based_dict

# Assuming the file path is 'family_graph.txt', which needs to be updated with the actual file path
file_path = 'family_graph.txt'  # Update this path
relations_dict = parse_family_relations_v3(file_path)
for relation, pairs in relations_dict.items():
    print(f"{relation}: {pairs}")


Mom: [['Alexis', 'Esha'], ['Rhea', 'Jasmine']]
Dad: [['Jayden', 'Esha'], ['Ram', 'Leo'], ['Ram', 'Jayden'], ['Leo', 'Jasmine']]
Brother: [['Michael', 'Esha']]
Sister: [['Esha', 'Michael']]
Married: [['Alexis', 'Jayden']]
Uncle: [['Leo', 'Michael'], ['Leo', 'Esha']]
Cousin: [['Esha', 'Jasmine']]


In [1]:
from graphviz import Digraph
from IPython.display import display

# Create a Digraph object
dot = Digraph(comment='Family Tree')
dot.attr(rankdir='TB')  # Top to Bottom graph

# Nodes and their styles
dot.attr('node', shape='ellipse')

# Adding family members based on the relationships
# Set the node only once, if not already set
added_nodes = set()

for relation, person_pairs in relations_dict.items():
    for pair in person_pairs:
        parent, child = pair
        if parent not in added_nodes:
            dot.node(parent, parent)
            added_nodes.add(parent)
        if child not in added_nodes:
            dot.node(child, child)
            added_nodes.add(child)

        if relation == 'parent':
            # Add edge from parent to child
            dot.edge(parent, child)
        elif relation == 'Married':
            # Add edge for marriage (bidirectional with no arrows)
            dot.edge(parent, child, dir='none', constraint='false')
        elif relation == 'Brother':
            dot.edge('Alexis', 'Michael')  
            dot.edge('Jayden', 'Michael')  
display(dot)

NameError: name 'relations_dict' is not defined

In [34]:
def find_ancestors_v2(person, relation_based_dict):
    ancestors = []
    ancestors.append(person)
    
    def add_parents_of_sibling(sibling):
        """Add parents based on a sibling's known parents."""
        for relation in ['Mom', 'Dad']:
            if relation in relation_based_dict:
                for pair in relation_based_dict[relation]:
                    # If the sibling is found as a child in a parent relation
                    if sibling == pair[1]:
                        # Add this parent for the original person as well
                        if pair[0] not in ancestors:
                            ancestors.append(pair[0])
                            recurse_find(pair[0])

    def recurse_find(child):
        # Initially look for 'Mom' and 'Dad' relations directly
        direct_parent_found = False
        for relation in ['Mom', 'Dad']:
            if relation in relation_based_dict:
                for pair in relation_based_dict[relation]:
                    if child == pair[1] and pair[0] not in ancestors:
                        direct_parent_found = True
                        ancestors.append(pair[0])
                        recurse_find(pair[0])

        # If no direct parent is found, look for siblings as a way to find parents
        if not direct_parent_found:
            for pair in relation_based_dict.get('Brother', []) + relation_based_dict.get('Sister', []):
                if child == pair[0]:  # If the person is listed as a sibling
                    add_parents_of_sibling(pair[1])
                elif child == pair[1]:  # Or the sibling relation is listed the other way
                    add_parents_of_sibling(pair[0])

    recurse_find(person)
    return ancestors



person = 'Esha'
# Test the updated function with an example
ancestors_of_person = find_ancestors_v2(person, relations_dict)
print(f"Ancestors of {person}: {ancestors_of_person}")


Ancestors of Esha: ['Esha', 'Alexis', 'Jayden', 'Ram']


In [28]:
def find_lca_v2(a, b, relation_based_dict):
    # Use the updated version of find_ancestors function
    ancestors_a = find_ancestors_v2(a, relation_based_dict)
    ancestors_b = find_ancestors_v2(b, relation_based_dict)
    
    # Find the first common ancestor in the list of ancestors for 'a' that also appears in 'b's ancestors
    # This relies on the ancestors being returned in order from closest to furthest
    lca = next((ancestor for ancestor in ancestors_a if ancestor in ancestors_b), None)
    return lca

# Assuming `relation_based_dict` is already defined and populated as shown previously
# Example use of the modified find_lca function

person1 = 'Alexis'
person2 = 'Esha'

lca = find_lca_v2(person1, person2, relations_dict)
print(f"The lowest common ancestor (LCA) of {person1} and {person2} is: {lca}")


The lowest common ancestor (LCA) of Alexis and Esha is: Alexis


In [48]:
# More detailed function 
def calculate_relationship(person_a, person_b, relation_based_dict):
    # First, check if their relationship already exists in the dictionary
    for relation, pairs in relation_based_dict.items():
        if [person_a, person_b] in pairs:
            return relation
    # Find the LCA for both individuals
    lca = find_lca_v2(person_a, person_b, relation_based_dict)
    if not lca:
        return "No common ancestors found, so no direct familial relationship."
    
    # Find the distance from each person to the LCA
    distance_a_to_lca = find_distance_to_lca(person_a, lca, relation_based_dict)
    distance_b_to_lca = find_distance_to_lca(person_b, lca, relation_based_dict)
    
    # Calculate the specific relationship based on the distances
    return calculate_detailed_relationship(distance_a_to_lca, distance_b_to_lca)

def find_distance_to_lca(person, lca, relation_based_dict):
    ancestors = find_ancestors_v2(person, relation_based_dict)
    distance = 0
    for ancestor in ancestors:
        distance += 1
        if ancestor == lca:
            return distance
    return -1  # Should not happen if LCA is correctly found

def calculate_detailed_relationship(distance_a, distance_b):
    # This function now needs to incorporate the detailed relationship logic
    # Direct relationships

    if distance_a == 1 and distance_b == 0:
        return "Parent"
    elif distance_a == 0 and distance_b == 1:
        return "Child"
    elif distance_a == 1 and distance_b == 1:
        return "Sibling"
    
    # Checking for the generation gap and determining the relationship accordingly
    generation_gap = abs(distance_a - distance_b)
    if generation_gap == 0:
        if distance_a == 2:
            return "Cousin"
    elif generation_gap == 1:
        if distance_b > distance_a:
            # Person A is the older generation
            if distance_b == 1:
                return "Parent"
            elif distance_b == 2:
                return "Uncle/Aunt"
        else:
            # Person B is the older generation
            if distance_a == 2:
                return "Child"
            elif distance_a == 3:
                return "Niece/Nephew"
    elif generation_gap > 1:
        # More distant ancestor-descendant relationship
        if distance_b > distance_a:
            return "Grandfather"
        else:
            return "Grandchild"

    return "Distant Relative or Complex Relationship"

def ordinal(n):
    """Convert an integer into its ordinal representation."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return str(n) + suffix


person1 = 'Esha'
person2 = 'Alexis'
# Test the function with an example
relationship = calculate_relationship(person1, person2, relations_dict)
print(f" {person1} is {person2}'s {relationship}")


 Esha is Alexis's Child


In [51]:
query = input("")
query = query.split()
person1 = query[2]
person2 = query[-1]

relationship = calculate_relationship(person1,person2,relations_dict)

print(f"{person1} is {person2}'s {relationship}")

Michael is Esha's Brother
